In [ ]:
!pip3 install --upgrade nvitop

In [1]:
from nvitop import Device

devices = Device.all()  # or Device.cuda.all()
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]
    sorted_pids = sorted(processes)

    print(device)
    print(f'  - Fan speed:       {device.fan_speed()}%')
    print(f'  - Temperature:     {device.temperature()}C')
    print(f'  - GPU utilization: {device.gpu_utilization()}%')
    print(f'  - Total memory:    {device.memory_total_human()}')
    print(f'  - Used memory:     {device.memory_used_human()}')
    print(f'  - Free memory:     {device.memory_free_human()}')
    print(f'  - Processes ({len(processes)}): {sorted_pids}')
    for pid in sorted_pids:
        print(f'    - {processes[pid]}')
    print('-' * 120)

PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB)
  - Fan speed:       41%
  - Temperature:     35C
  - GPU utilization: 21%
  - Total memory:    24.00GiB
  - Used memory:     786.3MiB
  - Free memory:     22.92GiB
  - Processes (16): [2064, 2420, 5684, 5948, 13048, 14192, 15304, 15352, 16756, 19900, 21096, 21316, 21616, 22212, 22708, 23544]
    - GpuProcess(pid=2064, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2064, name='dwm.exe', status='running', started='09:11:52'))
    - GpuProcess(pid=2420, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2420, name='msedgewebview2.exe', status='stopped', started='09:13:01'))
    - GpuProcess(pid=5684, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.proces

In [2]:
import time

from nvitop import Device, GpuProcess, NA, colored

print(colored(time.strftime('%a %b %d %H:%M:%S %Y'), color='red', attrs=('bold',)))

devices = Device.cuda.all()  # or `Device.all()` to use NVML ordinal instead
separator = False
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]

    print(colored(str(device), color='green', attrs=('bold',)))
    print(colored('  - Fan speed:       ', color='blue', attrs=('bold',)) + f'{device.fan_speed()}%')
    print(colored('  - Temperature:     ', color='blue', attrs=('bold',)) + f'{device.temperature()}C')
    print(colored('  - GPU utilization: ', color='blue', attrs=('bold',)) + f'{device.gpu_utilization()}%')
    print(colored('  - Total memory:    ', color='blue', attrs=('bold',)) + f'{device.memory_total_human()}')
    print(colored('  - Used memory:     ', color='blue', attrs=('bold',)) + f'{device.memory_used_human()}')
    print(colored('  - Free memory:     ', color='blue', attrs=('bold',)) + f'{device.memory_free_human()}')
    if len(processes) > 0:
        processes = GpuProcess.take_snapshots(processes.values(), failsafe=True)
        processes.sort(key=lambda process: (process.username, process.pid))

        print(colored(f'  - Processes ({len(processes)}):', color='blue', attrs=('bold',)))
        fmt = '    {pid:<5}  {username:<8} {cpu:>5}  {host_memory:>8} {time:>8}  {gpu_memory:>8}  {sm:>3}  {command:<}'.format
        print(colored(fmt(pid='PID', username='USERNAME',
                          cpu='CPU%', host_memory='HOST-MEM', time='TIME',
                          gpu_memory='GPU-MEM', sm='SM%',
                          command='COMMAND'),
                      attrs=('bold',)))
        for snapshot in processes:
            print(fmt(pid=snapshot.pid,
                      username=snapshot.username[:7] + ('+' if len(snapshot.username) > 8 else snapshot.username[7:8]),
                      cpu=snapshot.cpu_percent, host_memory=snapshot.host_memory_human,
                      time=snapshot.running_time_human,
                      gpu_memory=(snapshot.gpu_memory_human if snapshot.gpu_memory_human is not NA else 'WDDM:N/A'),
                      sm=snapshot.gpu_sm_utilization,
                      command=snapshot.command))
    else:
        print(colored('  - No Running Processes', attrs=('bold',)))

    if separator:
        print('-' * 120)
    separator = True

Wed May 14 10:29:13 2025
CudaDevice(cuda_index=0, nvml_index=0, name="NVIDIA TITAN RTX", total_memory=24.00GiB)
  - Fan speed:       41%
  - Temperature:     35C
  - GPU utilization: 23%
  - Total memory:    24.00GiB
  - Used memory:     830.2MiB
  - Free memory:     22.87GiB
  - Processes (16):
    PID    USERNAME  CPU%  HOST-MEM     TIME   GPU-MEM  SM%  COMMAND
    2064   DWM-1      6.9  77.61MiB  1:17:21  WDDM:N/A  N/A  dwm.exe
    2420   user       0.0  115.5MiB  1:16:12  WDDM:N/A  N/A  "C:\Program Files (x86)\Microsoft\EdgeWebView\Application\136.0.3240.64\msedgewebview2.exe" --type=gpu-process --noerrdialogs --user-data-dir=C:\Users\user\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\LocalState\EBWebView --webview-exe-name=SearchApp.exe --webview-exe-version=10.0.19041.5794 --embedded-browser-webview=1 --gpu-preferences=UAAAAAAAAADgAAAEAAAAAAAAAAAAAAAAAABgAAEAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAEAAAAAAAAAAIAAAAAAAAAAgAAAAAAAAA --always-read-main

In [3]:
from nvitop import Device

devices = Device.all()  # or `Device.cuda.all()` to use CUDA ordinal instead
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]
    sorted_pids = sorted(processes.keys())

    print(device)
    print(f'  - Fan speed:       {device.fan_speed()}%')
    print(f'  - Temperature:     {device.temperature()}C')
    print(f'  - GPU utilization: {device.gpu_utilization()}%')
    print(f'  - Total memory:    {device.memory_total_human()}')
    print(f'  - Used memory:     {device.memory_used_human()}')
    print(f'  - Free memory:     {device.memory_free_human()}')
    print(f'  - Processes ({len(processes)}): {sorted_pids}')
    for pid in sorted_pids:
        print(f'    - {processes[pid]}')
    print('-' * 120)

PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB)
  - Fan speed:       40%
  - Temperature:     35C
  - GPU utilization: 20%
  - Total memory:    24.00GiB
  - Used memory:     813.2MiB
  - Free memory:     22.89GiB
  - Processes (16): [2064, 2420, 5684, 5948, 13048, 14192, 15304, 15352, 16756, 19900, 21096, 21316, 21616, 22212, 22708, 23544]
    - GpuProcess(pid=2064, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2064, name='dwm.exe', status='running', started='09:11:52'))
    - GpuProcess(pid=2420, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2420, name='msedgewebview2.exe', status='stopped', started='09:13:01'))
    - GpuProcess(pid=5684, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.proces

In [4]:
from nvitop import Device

devices = Device.all()  # or `Device.cuda.all()` to use CUDA ordinal instead
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]
    sorted_pids = sorted(processes.keys())

    print(device)
    print(f'  - Fan speed:       {device.fan_speed()}%')
    print(f'  - Temperature:     {device.temperature()}C')
    print(f'  - GPU utilization: {device.gpu_utilization()}%')
    print(f'  - Total memory:    {device.memory_total_human()}')
    print(f'  - Used memory:     {device.memory_used_human()}')
    print(f'  - Free memory:     {device.memory_free_human()}')
    print(f'  - Processes ({len(processes)}): {sorted_pids}')
    for pid in sorted_pids:
        print(f'    - {processes[pid]}')
    print('-' * 120)

PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB)
  - Fan speed:       40%
  - Temperature:     36C
  - GPU utilization: 23%
  - Total memory:    24.00GiB
  - Used memory:     862.1MiB
  - Free memory:     22.84GiB
  - Processes (16): [2064, 2420, 5684, 5948, 13048, 14192, 15304, 15352, 16756, 19900, 21096, 21316, 21616, 22212, 22708, 23544]
    - GpuProcess(pid=2064, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2064, name='dwm.exe', status='running', started='09:11:52'))
    - GpuProcess(pid=2420, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.process.HostProcess(pid=2420, name='msedgewebview2.exe', status='stopped', started='09:13:01'))
    - GpuProcess(pid=5684, gpu_memory=N/A, type=C+G, device=PhysicalDevice(index=0, name='NVIDIA TITAN RTX', total_memory=24.00GiB), host=nvitop.api.proces

In [5]:
import time

from nvitop import Device, GpuProcess, NA, colored

print(colored(time.strftime('%a %b %d %H:%M:%S %Y'), color='red', attrs=('bold',)))

devices = Device.cuda.all()  # or `Device.all()` to use NVML ordinal instead
separator = False
for device in devices:
    processes = device.processes()  # type: Dict[int, GpuProcess]

    print(colored(str(device), color='green', attrs=('bold',)))
    print(colored('  - Fan speed:       ', color='blue', attrs=('bold',)) + f'{device.fan_speed()}%')
    print(colored('  - Temperature:     ', color='blue', attrs=('bold',)) + f'{device.temperature()}C')
    print(colored('  - GPU utilization: ', color='blue', attrs=('bold',)) + f'{device.gpu_utilization()}%')
    print(colored('  - Total memory:    ', color='blue', attrs=('bold',)) + f'{device.memory_total_human()}')
    print(colored('  - Used memory:     ', color='blue', attrs=('bold',)) + f'{device.memory_used_human()}')
    print(colored('  - Free memory:     ', color='blue', attrs=('bold',)) + f'{device.memory_free_human()}')
    if len(processes) > 0:
        processes = GpuProcess.take_snapshots(processes.values(), failsafe=True)
        processes.sort(key=lambda process: (process.username, process.pid))

        print(colored(f'  - Processes ({len(processes)}):', color='blue', attrs=('bold',)))
        fmt = '    {pid:<5}  {username:<8} {cpu:>5}  {host_memory:>8} {time:>8}  {gpu_memory:>8}  {sm:>3}  {command:<}'.format
        print(colored(fmt(pid='PID', username='USERNAME',
                          cpu='CPU%', host_memory='HOST-MEM', time='TIME',
                          gpu_memory='GPU-MEM', sm='SM%',
                          command='COMMAND'),
                      attrs=('bold',)))
        for snapshot in processes:
            print(fmt(pid=snapshot.pid,
                      username=snapshot.username[:7] + ('+' if len(snapshot.username) > 8 else snapshot.username[7:8]),
                      cpu=snapshot.cpu_percent, host_memory=snapshot.host_memory_human,
                      time=snapshot.running_time_human,
                      gpu_memory=(snapshot.gpu_memory_human if snapshot.gpu_memory_human is not NA else 'WDDM:N/A'),
                      sm=snapshot.gpu_sm_utilization,
                      command=snapshot.command))
    else:
        print(colored('  - No Running Processes', attrs=('bold',)))

    if separator:
        print('-' * 120)
    separator = True

Wed May 14 10:31:51 2025
CudaDevice(cuda_index=0, nvml_index=0, name="NVIDIA TITAN RTX", total_memory=24.00GiB)
  - Fan speed:       41%
  - Temperature:     36C
  - GPU utilization: 22%
  - Total memory:    24.00GiB
  - Used memory:     864.6MiB
  - Free memory:     22.84GiB
  - Processes (16):
    PID    USERNAME  CPU%  HOST-MEM     TIME   GPU-MEM  SM%  COMMAND
    2064   DWM-1      7.3  77.61MiB  1:19:58  WDDM:N/A  N/A  dwm.exe
    2420   user       0.0  115.4MiB  1:18:49  WDDM:N/A  N/A  "C:\Program Files (x86)\Microsoft\EdgeWebView\Application\136.0.3240.64\msedgewebview2.exe" --type=gpu-process --noerrdialogs --user-data-dir=C:\Users\user\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\LocalState\EBWebView --webview-exe-name=SearchApp.exe --webview-exe-version=10.0.19041.5794 --embedded-browser-webview=1 --gpu-preferences=UAAAAAAAAADgAAAEAAAAAAAAAAAAAAAAAABgAAEAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAEAAAAAAAAAAIAAAAAAAAAAgAAAAAAAAA --always-read-main

In [ ]:
import datetime
import time

import pandas as pd

from nvitop import ResourceMetricCollector

collector = ResourceMetricCollector(root_pids={1}, interval=2.0)  # log all devices and all GPU processes
df = pd.DataFrame()

with collector(tag='resources'):
    for _ in range(60):
        # Do something
        time.sleep(60)

        metrics = collector.collect()
        df_metrics = pd.DataFrame.from_records(metrics, index=[len(df)])
        df = pd.concat([df, df_metrics], ignore_index=True)
        # Flush to CSV file ...

df.insert(0, 'time', df['resources/timestamp'].map(datetime.datetime.fromtimestamp))
df.to_csv('results.csv', index=False)